In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time, csv
import pandas as pd
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.relative_locator import locate_with

In [12]:
driver = webdriver.Chrome(ChromeDriverManager().install())

C:\Users\panag\AppData\Local\Temp/ipykernel_2272/1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


## 1. Football

In [13]:
football_url = 'https://en.stoiximan.gr/sport/soccer/'
driver.get(football_url)
cookies = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.ID, 'onetrust-accept-btn-handler'))).click()

In [14]:
# Close Log in pop up window
x_button = driver.find_element(By.CSS_SELECTOR, '[class="sb-modal__close__btn uk-modal-close-default uk-icon uk-close"]').click()

In [25]:
# Click 24 hours button
button_24 = driver.find_element(By.CSS_SELECTOR, '[class="events-tabs-container__tab__item__button GTM-24"]').click()

In [38]:
### Scroll down the page and wait all values to load ###

# Get initial page height
last_height = driver.execute_script('window,scrollTo(0,document.body.scrollHeight)')

while True:
    # Wait for new values to load
    time.sleep(3)
    # Calculate the new height of the page and check if it has changed
    new_height = driver.execute_script('window,scrollTo(0,document.body.scrollHeight)')
    if new_height == last_height:
        # if the page height hasn't changed, exit the loop
        break
    else:
        # If the page height has changed, update the last height and continue scrolling
        last_height = new_height


# Wait for the dailyCoupon_body element to be present on the page
wait = WebDriverWait(driver, 10)
# By deafult is the football first
daily_coupon_body = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '[class="grid__column grid__column--fluid grid__column--main"]')))

# Extract footbal text
football_string = daily_coupon_body.text

In [40]:
football_string

"Home Soccer Next 24 Hours Full Coupon\nSoccer - Complete Coupon\nAll\n3 hours\n12 hours\n24 hours\nBy start time\nBy Competition\nSoccer - Matches in the next 24 hours\nMatches\n1\nX\n2\nO/U 2.5\nGG/NG\n  23/03\n23:00\nAlgeria\nNiger\n1.07\n9.00\n30.00\nO\n1.42\nU\n2.65\nYes\n3.65\nNo\n1.24\n134\n23/03\n23:00\nReal Espana II\nReal Sociedad II\n3.10\n3.60\n2.02\nO\n1.62\nU\n2.15\n16\n23/03\n23:30\nFrench Guyana\nDominican Republic\n1.83\n3.60\n3.75\nO\n1.75\nU\n1.93\nYes\n1.70\nNo\n2.00\n35\n24/03\n00:00\nBrusque SC\nConcordia\n1st leg: 0-0\n1.78\n3.00\n5.20\nO\n2.37\nU\n1.50\nYes\n2.15\nNo\n1.60\n38\n24/03\n00:00\nAS Arapiraquense AL\nCE Olhodaguense AL\n1.05\n8.50\n30.00\n4\n24/03\n00:00\nGoiás GO\nBrasiliense\n1st leg: 0-0\n1.37\n4.35\n7.70\nO\n1.78\nU\n1.91\nYes\n2.07\nNo\n1.65\n38\n24/03\n00:00\nStar Arafat\nVidad Arafat\n4.00\n3.30\n1.83\nO\n2.00\nU\n1.72\n16\n24/03\n00:00\nAlianza FC (W)\nTauro FC (W)\n26.00\n11.00\n1.03\n14\n24/03\n00:15\nAmerica de Cali (W)\nAtletico Bucaraman

In [41]:
# Create list from the initial string
initial_list = football_string.split('\n')

In [42]:
initial_list

['Home Soccer Next 24 Hours Full Coupon',
 'Soccer - Complete Coupon',
 'All',
 '3 hours',
 '12 hours',
 '24 hours',
 'By start time',
 'By Competition',
 'Soccer - Matches in the next 24 hours',
 'Matches',
 '1',
 'X',
 '2',
 'O/U 2.5',
 'GG/NG',
 '  23/03',
 '23:00',
 'Algeria',
 'Niger',
 '1.07',
 '9.00',
 '30.00',
 'O',
 '1.42',
 'U',
 '2.65',
 'Yes',
 '3.65',
 'No',
 '1.24',
 '134',
 '23/03',
 '23:00',
 'Real Espana II',
 'Real Sociedad II',
 '3.10',
 '3.60',
 '2.02',
 'O',
 '1.62',
 'U',
 '2.15',
 '16',
 '23/03',
 '23:30',
 'French Guyana',
 'Dominican Republic',
 '1.83',
 '3.60',
 '3.75',
 'O',
 '1.75',
 'U',
 '1.93',
 'Yes',
 '1.70',
 'No',
 '2.00',
 '35',
 '24/03',
 '00:00',
 'Brusque SC',
 'Concordia',
 '1st leg: 0-0',
 '1.78',
 '3.00',
 '5.20',
 'O',
 '2.37',
 'U',
 '1.50',
 'Yes',
 '2.15',
 'No',
 '1.60',
 '38',
 '24/03',
 '00:00',
 'AS Arapiraquense AL',
 'CE Olhodaguense AL',
 '1.05',
 '8.50',
 '30.00',
 '4',
 '24/03',
 '00:00',
 'Goiás GO',
 'Brasiliense',
 '1st leg: 0-0

In [ ]:
# Create list from the initial string
initial_list = football_string.split('\n')

# Remove first elements of the list not needed
remove_elements = ['Daily coupon','Football','Tennis','Basketball',
                   '24 hours','12 hours','3 hours','Popular First','SO']

football_list = [x for x in initial_list if x not in remove_elements]